In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install transformers
#! git clone https://github.com/jmunozmendi/SNGP

In [2]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
import numpy as np

In [3]:
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F
from torch import nn
import torch
import os
import torchvision.datasets as dset
import torchvision.transforms as transforms
#import gpytorch
import math
import tqdm

# Load dataset

In [4]:
import pandas as pd

dataset = pd.read_csv('dataset.tsv', sep='\t')

dataset['text'] = dataset['text'].astype(str)

dataset.head()

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def preprocess_function(examples):
    return tokenizer(examples,
                           padding='max_length',  # Pad to max_length
                           truncation=True,       # Truncate to max_length
                           max_length=200)        # Specify the max length


In [7]:
dataset['tokenized_text'] = dataset['text'].apply(lambda x: preprocess_function(x))

In [8]:
input_ids = dataset['tokenized_text'].apply(lambda x: x['input_ids'])
attention_masks = dataset['tokenized_text'].apply(lambda x: x['attention_mask'])

In [9]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


input_ids_tensor = torch.tensor(input_ids)
attention_masks_tensor = torch.tensor([item for item in attention_masks])
labels_tensor = torch.tensor(dataset['label'].values)

# Custom Dataset class
class DistilBERTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Return a tuple where the first element is a tuple of input_ids and attention_mask,
        # and the second element is labels
        return (self.input_ids[idx], self.attention_masks[idx]), self.labels[idx]


dataset = DistilBERTDataset(input_ids_tensor, attention_masks_tensor, labels_tensor)

# Split dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = train_test_split(dataset, train_size=train_size, test_size=test_size, stratify=labels_tensor)


## Creating the DistilBERT Model

In [10]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self, num_outputs = 768):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, num_outputs)


    def forward(self, X):
        (input_ids, attention_mask) = X

        output_1 = self.l1(input_ids=input_ids.long(), attention_mask=attention_mask.long())
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        output = self.pre_classifier(pooler)

        return output


num_inputs_gp = 768
feature_extractor = DistillBERTClass(num_outputs = num_inputs_gp)

## Creating the GP Layer

In the next cell, we create the layer of Gaussian process models that are called after the neural network. In this case, we'll be using one GP per feature, as in the SV-DKL paper. The outputs of these Gaussian processes will the be mixed in the softmax likelihood.

In [11]:
from SNGP.sngp import SNGP

## Creating the full SVDKL Model

With both the DenseNet feature extractor and GP layer defined, we can put them together in a single module that simply calls one and then the other, much like building any Sequential neural network in PyTorch. This completes defining our DKL model.

In [12]:
model = SNGP(out_features=2, backbone=feature_extractor, backbone_output_features = num_inputs_gp, num_inducing=128, momentum = 0)


In [13]:
print(model)

SNGP(
  (rff): Sequential(
    (0): DistillBERTClass(
      (l1): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
              (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

## Defining Training and Testing Code

Next, we define the basic optimization loop and testing code. This code is entirely analogous to the standard PyTorch training loop. We create a `torch.optim.SGD` optimizer with the parameters of the neural network on which we apply the standard amount of weight decay suggested from the paper, the parameters of the Gaussian process (from which we omit weight decay, as L2 regualrization on top of variational inference is not necessary), and the mixing parameters of the Softmax likelihood.

We use the standard learning rate schedule from the paper, where we decrease the learning rate by a factor of ten 50% of the way through training, and again at 75% of the way through training.

In [14]:
model.train_model(dataset=train_dataset, epochs=10, batch_size=32, weight_decay=0.01, lr = 2e-5)

[INFO] Training in progress... 10%  Mean Loss...2.325909764079724
[INFO] Training in progress... 20%  Mean Loss...1.963748667831138
[INFO] Training in progress... 30%  Mean Loss...1.6684540463453632
[INFO] Training in progress... 40%  Mean Loss...1.498215650174751
[INFO] Training in progress... 50%  Mean Loss...1.3475671955021256
[INFO] Training in progress... 60%  Mean Loss...1.223180852777793
[INFO] Training in progress... 70%  Mean Loss...1.1167957444399415
[INFO] Training in progress... 80%  Mean Loss...1.0401839040763723
[INFO] Training in progress... 90%  Mean Loss...0.964212639943732
[INFO] Training in progress... 100%  Mean Loss...0.9009173690489275
Standard inversion


In [15]:
info = model.predict(dataset=test_dataset, batch_size=256)

[INFO] Inference in progress... 50%
[INFO] Inference in progress... 100%


In [16]:
test_x, test_y = zip(*test_dataset)

In [17]:
test_x, test_y = np.array(test_x), np.array(test_y)

<ipython-input-17-a78989f4ec14>:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  test_x, test_y = np.array(test_x), np.array(test_y)
<ipython-input-17-a78989f4ec14>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_x, test_y = np.array(test_x), np.array(test_y)


In [18]:
result = info.decision == test_y

In [19]:
print("Accuracy: ", result.sum() / len(result))

Accuracy:  0.9661016949152542


In [20]:
info

SNGPInfo(decision=array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1,